In [1]:
import numpy as np
import pickle
from pathlib import Path
from os import walk
import nrrd
from sklearn.preprocessing import QuantileTransformer

## Helper code for reading all nrrds from folder

In [6]:
def clip_percentiles(volume, percentile_clip = [0.001, 0.999]):
    """
    This method calculates and clips the volume by 
    the upper and lower bounderies of the lower and 
    upper percentiles of the provided tensor

    input: array
    return: volume, dict
    """
    lower = np.percentile(volume, percentile_clip[0])
    upper = np.percentile(volume, percentile_clip[1])
    percentiles = {"upper": upper, "lower": lower}
    clipped = np.clip(volume, lower, upper)
    return clipped, percentiles

def quantile_transform_data(clipped_volume):
    """
    This method quantile transformed the clipped data
    this function returns the transformed data and 
    the transformer for further use.
    """
    sk_fmt = clipped_volume.reshape(-1, 1)
    quantile_transformer = QuantileTransformer(
    output_distribution='uniform',
    n_quantiles=max(min(sk_fmt.shape[0] // 100, 1000), 10), # Heuristic for n_quantiles
    random_state=42
    )
    quantile_transformer.fit(sk_fmt)
    transformed_volume = quantile_transformer.transform(sk_fmt)
    reverted_transformed_volume = transformed_volume.reshape(clipped_volume.shape)
    return quantile_transformer, reverted_transformed_volume

def save_transformer_and_percentiles(path, quantile_transformer, percentiles):
    """
    This method packages the necessary object for preprocessing the
    loose MRI sequences according to the training data.
    It requires a path out, the quantile transformer and the percentiles
    needed for clipping.

    returns: None
    """
    manifest = {"transformer": quantile_transformer,
                "lower": percentiles["lower"],
                "upper": percentiles["upper"]}
    
    with open(path, "+bw") as tr_out:
        pickle.dump(manifest, tr_out)

In [3]:
def get_seqs(path = "input_dir"):
    segmentation_files = []
    sequence_files = []

    # read files from disc
    for (root, direc, files) in walk(path):
        for file in files:
            # Just exvivo
            if not "invivo" in root or not "invivo" in direc or not "invivo" in file:
                # if we have a .nrrd segmentation
                if file.endswith("label.nrrd") or file.endswith("seg.nrrd"):
                    segmentation_files.append(Path(root)/Path(file))
                # if we have a sequence
                if not file.endswith("label.nrrd") and not file.endswith("seg.nrrd") and file.endswith(".nrrd"):
                    sequence_files.append(Path(root)/Path(file))

    # ensure correct sorting
    segmentation_files.sort(), sequence_files.sort()

    # load the sequences into a list
    sequence_files = [nrrd.read(file)[0] for file in sequence_files]

    # load the segmentations into a list
    segmentation_files = [nrrd.read(file)[0] for file in segmentation_files]
    return sequence_files, segmentation_files

def transpose(seqs, segs):
    new_seq = []
    new_seg = []
    for seq in seqs:
        if seq.shape[0] > seq.shape[1]:
            new_seq.append(seq.transpose((1, 0, 2)))
        elif seq.shape[0] < seq.shape[1]:
            new_seq.append(seq)
        elif seq.shape[0] == seq.shape[1]:
            new_seq.append(seq)
        else:
            print("Something wrong")

    for seg in segs:
        if seg.shape[0] > seg.shape[1]:
            new_seg.append(seg.transpose((1, 0, 2)))
        elif seg.shape[0] < seg.shape[1]:
            new_seg.append(seg)
        elif seg.shape[0] == seg.shape[1]:
            new_seg.append(seg)
        else:
            print("Something wrong")
    return new_seq, new_seg


def crop_to_size(seqs, segs, x, y):
    return seqs[:,0:x,0:y,:], segs[:,0:x,0:y,:]

def pad_with_zeros(seqs, segs, pad_dim=(128, 128, 31)):
    new_seq = []
    new_seg = []

    for seq in seqs:
        pad = np.zeros(pad_dim)
        pad[0:seq.shape[0], 0:seq.shape[1], 0:seq.shape[2]] = seq
        new_seq.append(pad)
    
    for seg in segs:
        pad = np.zeros(pad_dim)
        pad[0:seg.shape[0], 0:seg.shape[1], 0:seg.shape[2]] = seg
        new_seg.append(pad)

    return new_seq, new_seg

def allow_only_by(seqs, segs, size=(128, 256, 20)):
    seqs = [seq for seq in seqs if seq.shape == size]
    segs = [seg for seg in segs if seg.shape == size]
    return seqs, segs


def convert_to_tensor(seqs, segs):
    seqs = np.concatenate(seqs, axis=2)
    segs = np.concatenate(segs, axis=2)
    seqs = seqs.transpose((2, 0, 1))
    segs = segs.transpose((2, 0, 1))
    seqs, segs = np.expand_dims(seqs, axis=-1), np.expand_dims(segs, axis=-1)
    return seqs, segs

## This function is non-robust/ broken

# def normalize_by(seq, min_v=0, maximum=None, std=0, stds=1, mean=0, clip=None, clip_max=None):
#     if clip_max:
#         seq = seq.clip(seq, max=clip_max)
#     if clip:
#         seq = seq.clip(seq, max = mean + (std * stds))
#     if not maximum:
#         maximum = seq.max()
#         seq = seq / maximum
#         print(f"max = {maximum}")
#     if maximum:
#         seq = seq / maximum
#     return seq

def train_test_validation(volume):
    pct_80 = int(volume.shape[0] / 100 * 80)
    pct_90 = int((volume.shape[0] / 100 * 90))
    pct_100 = int(volume.shape[0])
    return volume[0:pct_80], volume[pct_80: pct_90], volume[pct_90:pct_100]

def write_pickle(obj, file):
    with open(file, 'b+w') as pout:
        pickle.dump(obj, pout)

# ASL

In [ ]:
input_dir = Path("../ROI/ROI_NEW/ASL")
output_file = Path("../data/pickles/ASL.pkl")
transformer_path = Path("../transformers/ASL_transformer.pkl")
seqs, segs = get_seqs(input_dir)
seqs, segs = pad_with_zeros(seqs, segs, pad_dim=(128, 128, 31))
seqs, segs = convert_to_tensor(seqs, segs)
seqs, percentiles = clip_percentiles(seqs, percentile_clip=[0.00001, 99.99999])
quantile_transformer_ASL, seqs = quantile_transform_data(seqs)
save_transformer_and_percentiles(transformer_path, quantile_transformer_ASL, percentiles)
seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
ASL_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(ASL_data, output_file)

In [ ]:
seq_train.shape, seg_train.shape

# DWI

In [ ]:
input_dir = Path("../ROI/ROI_NEW/DWI")
output_file = Path("../data/pickles/DWI.pkl")
transformer_path = Path("../transformers/DWI_transformer.pkl")

seqs, segs = get_seqs(input_dir)
seqs, segs = transpose(seqs, segs)
seqs, segs = allow_only_by(seqs, segs, size=(104, 128, 15))
seqs, segs = convert_to_tensor(seqs, segs)
seqs, segs = crop_to_size(seqs, segs, 96, 128)
seqs, percentiles = clip_percentiles(seqs, percentile_clip=[0.00001, 99.99999])
quantile_transformer_DWI, seqs = quantile_transform_data(seqs)
save_transformer_and_percentiles(transformer_path, quantile_transformer_DWI, percentiles)
seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
DWI_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(DWI_data, output_file)

# T1

In [ ]:
input_dir = Path("../ROI/ROI_NEW/T1/exvivo")
output_file = Path("../data/pickles/T1.pkl")
transformer_path = Path("../transformers/T1_transformer.pkl")

seqs, segs = get_seqs(input_dir)
seqs, segs = convert_to_tensor(seqs, segs)

seqs, percentiles = clip_percentiles(seqs, percentile_clip=[0.00001, 99.99999])
quantile_transformer_T1, seqs = quantile_transform_data(seqs)
save_transformer_and_percentiles(transformer_path, quantile_transformer_T1, percentiles)

seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
T1_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(T1_data, output_file)

# T2

In [4]:
input_dir = Path("../ROI/ROI_NEW/T2/exvivo")
output_file = Path("../data/pickles/T2.pkl")
transformer_path = Path("../transformers/T2_transformer.pkl")

seqs, segs = get_seqs(input_dir)
seqs, segs = transpose(seqs, segs)
seqs, segs = convert_to_tensor(seqs, segs)

seqs, percentiles = clip_percentiles(seqs, percentile_clip=[0.00001, 99.99999])
quantile_transformer_T2, seqs = quantile_transform_data(seqs)
save_transformer_and_percentiles(transformer_path, quantile_transformer_T2, percentiles)

seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
T2_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(T2_data, output_file)

# T2star

In [ ]:
input_dir = Path("../ROI/ROI_NEW/T2star")
output_file = Path("../data/pickles/T2star.pkl")

transformer_path = Path("../transformers/T2star_transformer.pkl")

seqs, segs = get_seqs(input_dir)
seqs, segs = allow_only_by(seqs, segs)
seqs, segs = transpose(seqs, segs)
seqs, segs = convert_to_tensor(seqs, segs)

seqs, percentiles = clip_percentiles(seqs, percentile_clip=[0.00001, 99.99999])
quantile_transformer_T2star, seqs = quantile_transform_data(seqs)
save_transformer_and_percentiles(transformer_path, quantile_transformer_T2star, percentiles)

seg_train, seg_test, seg_val = train_test_validation(segs)
seq_train, seq_test, seq_val = train_test_validation(seqs)
T2star_data = {"seg_test": seg_test,
            "seq_test": seq_test,
            "seg_train": seg_train,
            "seq_train": seq_train,
            "seg_val": seg_val,
            "seq_val": seq_val}
write_pickle(T2star_data, output_file)